<a href="https://colab.research.google.com/github/girish445ai/Convolutional_Neural_Networks/blob/main/Convolutional_Neural_Networks_PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

     |████████████████████████████████| 1.7 MB 10.4 MB/s 
     |████████████████████████████████| 144 kB 74.6 MB/s 
     |████████████████████████████████| 181 kB 58.5 MB/s 
     |████████████████████████████████| 63 kB 957 kB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=863795a3f4e3d0d968ccac56bd61387dfdfbd0cda84c5a5815be16026509d0a0
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import numpy as np
import keras 
import tensorflow as tf
import pandas as pd
import wandb
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback

In [ ]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_datapath='inaturalist_12K/train/'
test_datapath='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

# Q1 PART=A

In [ ]:
def cnn_model(layer1, layer2, layer3, layer4, layer5, F,bn, dropout):
  input_shape = (200,200,3) # input image  size 

  model = models.Sequential()
  
  # first layer
  model.add(Conv2D(layer1, (F, F), input_shape=input_shape,activation='relu',name='layer1'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  if bn == 1:
    model.add(BatchNormalization())
 
  model.add(Conv2D(layer2, (F, F),input_shape=input_shape,activation='relu',name='layer2'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  if bn == 1:
    model.add(BatchNormalization())

  model.add(Conv2D(layer3, (F, F),input_shape=input_shape,activation='relu',name='layer3'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  if bn == 1:
    model.add(BatchNormalization())

  model.add(Conv2D(layer4, (F, F),input_shape=input_shape,activation='relu',name='layer4'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  if bn == 1:
    model.add(BatchNormalization())

  model.add(Conv2D(layer5, (F, F),input_shape=input_shape,activation='relu',name='layer5'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(128,activation='relu')) 
  model.add(Dropout(dropout))

  model.add(Dense(10,activation='softmax'))

  return model

In [ ]:
wandb.init()

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
        
    },
    'parameters': {
        'dropout': {
            'values': [0.2]
        },
        'F_size': {
            'values': [3] 
        },
                
        'layer1': {
            'values': [16]
        },

        'layer2': {
            'values': [32]
        },

        
        'layer3': {
            'values': [64]
        },

        
        'layer4': {
            'values': [128]
        },

        
        'layer5': {
            'values': [256]
        },

        'batch_norm': {
            'values': [True]
        },
        'augmentation': {
            'values': [False]
        },
    }
}


sweep_id = wandb.sweep(sweep_config, entity='jyothiraditya', project="assignment2")



wandb.sweep(sweep_config)
config_defaults = {
            'epochs': 5,
            'learning_rate': 1e-2
            
          }

def execute():
    wandb.init(config = sweep_config)
    wandb.log({"images" : wandb.Image(plt)})
    config = wandb.config
    #wandb.run.name = "f_{}_hn_{}_bs_{}_ac_{}_learning_rate_{}_init_{}_".format(config.hiddenlayers, config.hiddennodes, config.batch_size , config.activation_func ,config.learning_rate, config.initializer)
    cnn_model(config.layer1, config.layer2, config.layer3, config.layer4, config.layer5, config.F_size,config.batch_norm, config.dropout)

wandb.agent(sweep_id, execute)

In [ ]:
def data_gen(train_datapath, test_datapath):
  train_gen = ImageDataGenerator(rescale=1./255,validation_split=0.1).flow_from_directory(
                                                     train_datapath,
                                                     target_size=(224, 224),
                                                     batch_size=5000,
                                                     class_mode='rgb',
                                                     subset='training',
                                                     seed=123,
                                                     label_mode='categorical')
  val_gen = ImageDataGenerator(rescale=1./255,validation_split=0.1).flow_from_directory(
                                                     train_datapath,
                                                     target_size=(224, 224),
                                                     batch_size=200,
                                                     class_mode='rgb',
                                                     subset='validation',
                                                     seed=123,
                                                     label_mode='categorical')
  
  AUTOTUNE = tf.data.AUTOTUNE

  train_gen = train_gen.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_gen = val_gen.cache().prefetch(buffer_size=AUTOTUNE) 
  


  x_train, y_train = next(train_gen)

  x_val, y_val = next(val_gen)

  train_test = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest').flow_from_directory(test_datapath,
                                                                           target_size=(224, 224),
                                                                           batch_size=1000,
                                                                           class_mode='rgb',
                                                                           seed=123)
  AUTOTUNE = tf.data.AUTOTUNE
  train_test = train_test.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

  x_test,y_test = next(train_test)

  return x_train, y_test, x_val, y_val, x_test, y_test